### Eksperymenty

**Modele**
- resnet50
- vgg13

**Zbiory danych**
- CIFAR100
- SVHN

**Metody**
- Random
- KMeans
- KMeansPurity
- KMeansDino
- KMeansDinoPurity

**Ustawienia**
- ratio = (0.1, 1, 10)
- epochs = 10
- batch_size = 512
- clip = 5.0
- num_clusters = 8
- eqsize = True
- min_purity = 0.1 (SVHN), 0.01 (CIFAR100)

**Nazewnictwo przebiegów**
W nawiasach kwadratowych podano wartość do wyboru ("jeden z ...") w nawiasch klamrowych wartości opcjonalne.
ID służy do rozróżniania eksperymentów w tej samej konfiguracji

- [r50,vgg13],ep:10,bs:512,clip:5.0,{nc:20},{eqsize},{mp:[0.1,0.01]},ID:[1,2,3,..]
- przykład: r50,ep:10,bs:512,clip:5.0,ID:1
- przykład: r50,ep:10,bs:512,clip:50,nc:20,eqsize,mp:0.1,ID:1

**Uwagi**
- Przy każdym eksperymencie nowy model (!)

In [ ]:
ID = 1

In [ ]:
import torch
import torchvision
from torchvision.models import resnet50, vgg13

from kiss.experiment import Experiment
from kiss.sampler import RandomSampler, KMeansSampler, KMeansPuritySampler, KMeansDinoSampler, KMeansPurityDinoSampler
from kiss.utils.configs import CONFIGS

transform = torchvision.transforms.ToTensor()
dataset_cifar100_tr = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform)
dataset_cifar100_te = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform)

dataset_svhn_tr = torchvision.datasets.CIFAR100(root='../data', train=True, download=True, transform=transform)
dataset_svhn_te = torchvision.datasets.CIFAR100(root='../data', train=False, download=True, transform=transform)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    RandomSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    RandomSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    save_clusters="../checkpoints/kmeans,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    save_clusters="../checkpoints/kmeans,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.01,ID:{ID}"

model = resnet50(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansPuritySampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.01,
    eqsize=True,
    save_clusters="../checkpoints/kmeans-purity,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"r50,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.05,ID:{ID}"

model = resnet50(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansPuritySampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.05,
    eqsize=True,
    save_clusters="../checkpoints/kmeans-purity,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    RandomSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    RandomSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0)
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    load_clusters="../checkpoints/kmeans,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansSampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    eqsize=True,
    load_clusters="../checkpoints/kmeans,svhn")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.1,ID:{ID}"

model = vgg13(num_classes=100)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_cifar100_tr,
    dataset_cifar100_te, 
    KMeansPuritySampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.1,
    eqsize=True,
    load_clusters="../checkpoints/kmeans-purity,cifar100")
    
experiment.run("../experiments", RUN_NAME)

In [ ]:
RUN_NAME = f"vgg13,ep:10,bs:512,clip:5.0,nc:8,eqsize,mp:0.05,ID:{ID}"

model = vgg13(num_classes=10)
model.to(torch.device(CONFIGS.torch.device))

experiment = Experiment(
    model, 
    dataset_svhn_tr,
    dataset_svhn_te, 
    KMeansPuritySampler,
    ratio=(0.1, 1, 10),
    epochs=10,
    batch_size=512,
    clip=5.0,
    num_clusters=8,
    min_purity=0.05,
    eqsize=True,
    load_clusters="../checkpoints/kmeans-purity,svhn")
    
experiment.run("../experiments", RUN_NAME)